Warning! <font color="red">Google collab has maximum session lengths of 1 hour, after 1 hour of running a jupyter notebook it will automatically shut down. This means that if you are training a model, you will lose all progress after 1 hour when you decide to train multiple agents in one session. Therefore, it is recommended to train one agent at a time and save the model weights immediately after training!</font>  

On average it takes about 10 minutes to train an agent for 3000 episodes.

## Accessing the datasets from Google Drive & cloning the Github repo

In [1]:
# connect to google drive to retrieve our preprocessed datasets.
from google.colab import drive
drive.mount('/content/drive')
import os

Mounted at /content/drive


In [6]:
# the processed datasets folder needs to be moved to the github data folder
!ls "drive/MyDrive/Thesis research project"

 events.parquet   events_sample.parquet  'processed datasets'


In [7]:
# Clone the github repo that contains the code of the trainig agent
!git clone https://github.com/meeslindhout/Master-Thesis-Project---Cold-Start-Recommender-System-Session-based.git
# you can ignore the lfs error.

Cloning into 'Master-Thesis-Project---Cold-Start-Recommender-System-Session-based'...
remote: Enumerating objects: 167, done.
remote: Counting objects: 100% (167/167), done.
remote: Compressing objects: 100% (107/107), done.
remote: Total 167 (delta 92), reused 121 (delta 48), pack-reused 0
Receiving objects: 100% (167/167), 172.86 KiB | 5.58 MiB/s, done.
Resolving deltas: 100% (92/92), done.
Error downloading object: data/retailrocket/unprocessed/events.zip (0f16dc1): Smudge error: Error downloading data/retailrocket/unprocessed/events.zip (0f16dc1cbba1efd8b0de07fac9b5f4acbf10e68096179f31201fe99244730747): batch response: This repository is over its data quota. Account responsible for LFS bandwidth should purchase more data packs to restore access.

Errors logged to /content/Master-Thesis-Project---Cold-Start-Recommender-System-Session-based/.git/lfs/logs/20240609T083954.327508889.log
Use `git lfs logs last` to view the log.
error: external filter 'git-lfs filter-process' failed
fatal

In [26]:
# At the moment our data folder does not contain the preprocessed data that is currently in our google drive. We need to copy it from google drive to the github repo
!ls Master-Thesis-Project---Cold-Start-Recommender-System-Session-based/data

events_sample.parquet  retailrocket  tmall


In [27]:
# Copy the processed data folder to the data folder in the github repo
!cp -r "/content/drive/MyDrive/Thesis research project/processed datasets" Master-Thesis-Project---Cold-Start-Recommender-System-Session-based/data

In [28]:
# Now, our data folder does contain the preprocessed data folder and we are ready to go
!ls Master-Thesis-Project---Cold-Start-Recommender-System-Session-based/data

 events_sample.parquet	'processed datasets'   retailrocket   tmall


In [29]:
# set the working directory to be our github repo
os.chdir('Master-Thesis-Project---Cold-Start-Recommender-System-Session-based')

In [32]:
# install required packages
!pip install wandb -qU

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 26.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 21.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 289.6/289.6 kB 26.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 7.6 MB/s eta 0:00:00


In [35]:
# load all required libraries
import pandas as pd
from src.recsys_rl import rl_recommender
import wandb
from google.colab import runtime

In [36]:
# login to wandb to monitor agent training
wandb.login()

<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

## Load the dataset

In [51]:
train = pd.read_csv('data/processed datasets/diginetica/interactions_train_tr.txt',
                    sep='\t')
display(train.head())
display(train.shape)

,Type,SessionId,ItemId,Time,Date,Datestamp,TimeO,ItemSupport
0,view,1,9654,1.462752e+09,2016-05-09,1.462752e+09,2016-05-09 00:01:15.848000+00:00,74
1,view,1,33043,1.462752e+09,2016-05-09,1.462752e+09,2016-05-09 00:02:53.912000+00:00,41
2,view,1,32118,1.462752e+09,2016-05-09,1.462752e+09,2016-05-09 00:04:03.569000+00:00,19
3,view,1,12352,1.462752e+09,2016-05-09,1.462752e+09,2016-05-09 00:05:29.870000+00:00,79
4,view,1,35077,1.462752e+09,2016-05-09,1.462752e+09,2016-05-09 00:06:30.072000+00:00,47


(102272, 8)

In [ ]:
# how many sessions are in the training set?
train['SessionId'].nunique()

In [52]:
# get unique values from type
train['Type'].unique()

array(['view', 'purchase'], dtype=object)

In [55]:
# replace view with 0, and purchase with 1
train['Type'].replace({'view': 0, 'purchase': 1}, inplace=True)

In [57]:
# initialize the model / agent
model = rl_recommender(
    n_history=5,
    reward_dict={0:3, 1:10},
    event_key='Type',
    mode='training',
    num_episodes=3000,
    batch_size=64,
    target_update_freq=1000,
    memory=100_000,
    learning_rate=0.0003,
    gamma=0.99,
    dataset_name='diginetica',
    custom_wandb_note=''
    )

In [58]:
model.fit(train=train)

Started converting train data to tensors...
Episode tensors created successfully
Initializing gym environment...
Initializing offline training agent...


wandb: Currently logged in as: meeslindhout (universityofappliedsciencesamsterdam). Use `wandb login --relogin` to force relogin


Started training loop...
Episode 1/3000 completed with reward: 27
Episode 2/3000 completed with reward: 3
Episode 3/3000 completed with reward: 21
Episode 4/3000 completed with reward: 6
Episode 5/3000 completed with reward: 6
Episode 6/3000 completed with reward: 30
Episode 7/3000 completed with reward: 15
Episode 8/3000 completed with reward: 3
Episode 9/3000 completed with reward: 9
Episode 10/3000 completed with reward: 3
Episode 11/3000 completed with reward: 6
Episode 12/3000 completed with reward: 9
Episode 13/3000 completed with reward: 18
Episode 14/3000 completed with reward: 18
Episode 15/3000 completed with reward: 16
Episode 16/3000 completed with reward: 6


KeyboardInterrupt: 

In [ ]:
# copy all trained agents to google drive
!cp -r "/content/Master-Thesis-Project---Cold-Start-Recommender-System-Session-based/trained agents/" "/content/drive/MyDrive/Thesis research project/"

In [ ]:
# unassign the runtime from google collab. 
# This is to prevent the runtime from being disconnected after 12 hours.
runtime.unassign()